## Restructure Attention Shift to have trials, sub-blocks and blocks

This script assumes that `as_preliminary_summary.ipynb` has been run.
This script assumes that the data is in BIDS format and that each BIDS events
file of the form `_events.tsv` has a corresponding events file with
suffix `_events_temp.tsv` that was previously dumped from the `EEG.set` files.
The new files will be saved in `_events_temp2.tsv` (BIDS).

In order to compare the events coming from the BIDS events files and those
from the EEG.set files, the script creates dictionaries of `key` to full path
for each type of file.  The `key` is of the form `sub-xxx_run-y` which
uniquely specify each event file in the dataset. If a dataset contains
multiple sessions for each subject, the `key` should include additional
parts of the file name to uniquely specify each subject.

In [3]:
from hed.tools.io_utils import get_file_list, make_file_dict
from hed.tools.data_utils import get_new_dataframe
bids_root_path = 'G:/AttentionShift/AttentionShiftExperiments'
name_suffix = "_events_temp2"
name_indices = (0, -2)
files_bids = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix=name_suffix)
dict_bids = make_file_dict(files_bids, indices=name_indices)
print(f"\n{len(list(dict_bids))} Attention shift event files")


49 Attention shift event files


In [4]:
for key, file in dict_bids.items():
    df = get_new_dataframe(file)

    ## Set the sub blocks
    sub_mask = df['event_code'].map(str).isin(['1', '2']).astype(int)
    sub_total = 0
    sub_list = list(sub_mask)
    for i, value in sub_mask.iteritems():
        if value:
            sub_total += sub_list[i]
        sub_list[i] = sub_total
    df['sub_block'] = sub_list
    pause_col = df['event_code'].map(str) == '202'
    df.loc[pause_col, 'sub_block'] = 'n/a'

    ## Set the trials
    stim_total = 0
    stim_mask = \
        df['event_code'].map(str).isin(['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']).astype(int)
    stim_list = list(stim_mask)
    for i, value in stim_list.iteritems():
        if value:
            stim_total += stim_list[i]
        stim_mask.iloc[i] = stim_total
    df['trial'] = sub_list
    df.loc[pause_col, 'trial'] = 'n/a'
    print("to here")
       # df_new.to_csv(eeg_corrected, sep='\t', index=False)
    break

to here
